### Auto-Encoders
![](ae.png) 

In this exercise, we will implement an MLP based autoencoder to work with the lung data. 

Autoencoders such as the ones we will implement today can be thought to be non-linear PCAs. 

We begin by preparing the data:

1. Load the numpy arrays and labels
2. Create the Torch dataset, dataloaders
3. Reuse the training script from our MLP session
4. Code the Autoencoder!

In [ ]:
### Load necessary modules
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from IPython.display import clear_output

In [ ]:
# Load the data
data = pd.read_pickle('data.pkl')
# Load labels from the csv file
Y = data.type_num.values ###########
# Convert the labels from 0-1 to -1/+1
# Load the data array
X = data.pdf
X = np.stack(X,axis=0) 

In [ ]:
class pdfData(Dataset):
    def __init__(self,data,target,normalize=True):
        super().__init__()
        
        ## Load from numpy
        self.data = torch.FloatTensor(data) ############
        
        ### Add a channel dimension
        self.data = self.data.unsqueeze(1)
        
        ### Normalize intensities to be between 0-1
        if normalize:
            self.data = self.data/self.data.max()
        
        ## Make a tensor target
        self.target = torch.FloatTensor(target)

    def __len__(self):
        ### Method to return number of data points
        return len(self.target)
    
    def __getitem__(self,index):
        ### Method to fetch indexed element
        return self.data[index], self.target[index].type(torch.LongTensor)

In [ ]:
## Make the dataset by passing the numpy arrays!
dataset = pdfData(data=X,target=Y)

In [ ]:
#### Make training, validation and test sets
from torch.utils.data import Subset
#### We can use a random split function to make data subsets
N = len(dataset)
nTrain = 1600
nValid = 400
nTest = (N-nTrain-nValid)
train_set = Subset(dataset,torch.arange(nTrain))
valid_set = Subset(dataset,torch.arange(nTrain,nTrain+nValid))
test_set = Subset(dataset,torch.arange(nTrain+nValid,N))
random_split(dataset,[nTrain,nValid,nTest])
### Wrapping the datasets with DataLoader class opens up 
### batching options
batch_size = 4
train_loader = DataLoader(train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader(valid_set,batch_size=batch_size,shuffle=True)
test_loader  = DataLoader(test_set,batch_size=batch_size)

In [ ]:
### Fill in the relevant details 
import pdb
### Autoencoder class
class AE(nn.Module):
    def __init__(self,nIp,nhid,latent_dim):
        super(AE, self).__init__()
        
        ### Encoder layers
        self.enc1 = nn.Linear(nIp,nhid)
        self.enc2 = nn.Linear(nhid,nhid)
        self.enc3 = nn.Linear(nhid,nhid)
        self.enc4 = nn.Linear(nhid,nhid)
        self.enc5 = nn.Linear(nhid,latent_dim)
        ### Decoder layers
        self.dec1 = nn.Linear(latent_dim,nhid)
        self.dec2 = nn.Linear(nhid,nhid)
        self.dec3 = nn.Linear(nhid,nhid)
        self.dec4 = nn.Linear(nhid,nhid)
        self.dec5 = nn.Linear(nhid,nIp)

    def encode(self, x):
#         pdb.set_trace()
        ### Fill in the encoder
        ### by calling the corresponding layers
        ### initialized above. 
        ### You can use F.relu() to call the
        ### rectified linear unit activation function
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        z = (self.enc5(x))
        
        
        return z
    
    def decode(self, z):
        
        ### Fill in the decoder
        ### by calling the corresponding layers
        ### initialized above. 
        ### You can use torch.sigmoid() to call the
        ### sigmoid activation function
        x = F.relu(self.dec1(z))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        xHat = (self.dec5(x))
        
        return xHat

    def forward(self, x):
        ### Autoencoder returns the reconstruction 
        ### and latent representation
        z = self.encode(x)
        # decode z
        xHat = self.decode(z)
        return xHat,z 

In [ ]:
dataset[0][0].shape

In [ ]:
### Instantiate a model now! 
### Pass suitable parameters for our task
nFeat = dataset[0][0].shape[1]
model = AE(nIp=nFeat,nhid=32,latent_dim=2) #############
criterion = nn.MSELoss() ############ Loss function to be optimized. 
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3) #### Which optimizer do you want to use?

In [ ]:
#### Training and validation loop!

trLoss = []
trAcc = []
vlLoss = []
vlAcc = []

for epoch in range(100): ### Run the model for a certain number of iterations/epochs
    
    epLoss = 0
#     epAcc = 0
    
    for x, _ in train_loader: ### Fetch a batch of training inputs
        xIp = x.clone()
        xHat,z = model(x) ####### Obtain a prediction from the network
        loss = (criterion(xHat,xIp)) ######### Compute the loss between prediction and ground truth
        
        ### Backpropagation steps
        ### Clear old gradients
        optimizer.zero_grad()
        ### Compute the gradients wrt model weights
        loss.backward()
        ### Update the model parameters
        optimizer.step()
        
        epLoss += loss.item()

    trLoss.append(epLoss/len(train_loader))
    
    epLoss = 0
    
    for x, y in valid_loader: #### Fetch validation samples
        xIp = x.clone()
        xHat,z = model(x) ##########
        loss = (criterion(xHat,xIp))#######
        
        epLoss += loss.item()
    vlLoss.append(epLoss/len(valid_loader))
    
    
    if epoch % 5 == 0:
        clear_output(wait=True)
        print('Epoch: %03d, Tr. Loss: %.4f, Vl.Loss: %.4f'
              %(epoch,trLoss[-1],vlLoss[-1]))
        
        
        plt.plot(x[0].view(-1),label='Input')
        plt.plot(xHat[0].view(-1).detach().numpy(),label='Reconstruction',alpha=0.5)
        plt.legend()
        plt.show()
        
plt.plot(trLoss,label='training')
plt.plot(vlLoss,label='validation')

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

### Obtain the latent embeddings for validation set
embeddings = []
labels = []
for x, y in valid_loader: #### Fetch validation samples
    xHat,z = model(x) ##########
    embeddings.append(z.detach())
    labels.append(y)
    
embeddings = torch.stack(embeddings)
embeddings = embeddings.view(-1,embeddings.shape[-1]) 
labels = torch.stack(labels).view(-1)

# plot the first two dimensions
if embeddings.shape[-1] > 2:
#     embeddings = TSNE(n_components=2).fit_transform(embeddings)
     embeddings = PCA(n_components=2).fit_transform(embeddings)
    
    
plt.scatter(embeddings[:,0],embeddings[:,1],c=labels,cmap='rainbow')

In [ ]:
xNp = X[nTrain:(nValid+nTrain)]
yNp = Y[nTrain:(nTrain+nValid)]
pca_emb = PCA(n_components=2).fit_transform(xNp)
plt.scatter(pca_emb[:,0],pca_emb[:,1],c=yNp,cmap='rainbow')

In [ ]:
nTrain

**Auto-encoding input data**

It might come as no surprise but our predictions are the inputs themselves. Then what is the P for such a task?

1. Think of a reasonable criterion to optimize if the output has to be the same as the input
2. Implement the corresponding loss function
3. Set-up the training loop and plot one random validation sample every 10 iterations to see if the output is approaching the input


In [ ]:
####

### Data imputation

1. AEs can be applied to impute missing data
2. We will simulate this in this task by maksing off regions of the image


In [ ]:
#### 

### Extracting features from the AE

1. Use the trained Autoencoder to extract features
2. The _latent_ dimension representation is equivalent to transforming with PCA from yesterday's exercise
3. Obtain these representations for all data points in 2d and visualize them
4. Use them as input features to a classifier